<a href="https://colab.research.google.com/github/drew-walkerr/Diss_Detecting_Provider_Bias/blob/main/doubt_markers_regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import os
import re
import sys
import scipy
import Cython
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import matplotlib.pyplot as plt
import spacy
from spacy.lang.en import English
import numpy as np

from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive/')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [17]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


In [18]:

%cd /content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Doubt Markers/1_Data Prep
%ls

/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Doubt Markers/1_Data Prep
1_doubt_makers_word_embeddings.py
doubt_markers_lexicon_dev.ipynb
doubt_markers_regex.ipynb
doubt_word_list_round_1_dw_pruned.csv
doubt_word_list_round_2.csv
doubt_words_lexicon_stem_and_similar_round1.csv
expanded_misspellings.csv
lexicon_search_and_visualizations_subreddit.Rmd
trig-vectors-phrase.bin


BigQuery

In [19]:
#Replace 'project_id' with your BigQuery project ID

from google.cloud import bigquery
client = bigquery.Client(project='detecting-bias-mimic-iii')

icd_query = ('''SELECT *
                FROM physionet-data.mimiciii_clinical.diagnoses_icd
                LIMIT 1000''')

icd = client.query(icd_query).to_dataframe()



ICD codes

In [20]:


#Filter for ICD codes for 282.60-282.69, referring to sickle cell types w/wo crisis
#2824 for thalassemia w + w/o crisis (282.41-282.42)
#SCD: 2826,2824
#Chronic Pain 3382
#Opioid dependencies: 3040,3047 (combo),
#HIV/AIDS ^042$

icds_of_interest = icd[icd['ICD9_CODE'].str.contains('2826|2824|3040|3047|3382|^042$', na=False)]
# Issues here-- will cause more leading numbers
print(icds_of_interest.head())

icds_of_interest.info()

#patients_unique = icds_of_interest['SUBJECT_ID'].drop_duplicates()


     ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
784  176454       15872   139593        3     73382
889  196755       17664   123887        7     33829
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 784 to 889
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ROW_ID      2 non-null      Int64 
 1   SUBJECT_ID  2 non-null      Int64 
 2   HADM_ID     2 non-null      Int64 
 3   SEQ_NUM     2 non-null      Int64 
 4   ICD9_CODE   2 non-null      object
dtypes: Int64(4), object(1)
memory usage: 104.0+ bytes


Notes and patients load

In [73]:
note_query = ('''SELECT *
                FROM physionet-data.mimiciii_notes.noteevents
                LIMIT 500000''')

NOTES = client.query(note_query).to_dataframe()

peek_notes = NOTES.head()
print(peek_notes)

## Patients

patient_query = ('''SELECT *
                FROM physionet-data.mimiciii_clinical.patients''')

PATIENTS = client.query(patient_query).to_dataframe()

peek_patients = PATIENTS.head()
print(peek_patients)

biased_notes_patients_corpus = NOTES.merge(PATIENTS, on = 'SUBJECT_ID')

biased_notes_patients_corpus.info()
# Find unique chart types and decide to limit them
    # Remove Radiology, ECG, Respiratory, Echo notes
biased_notes_patients_corpus_filtered = biased_notes_patients_corpus[biased_notes_patients_corpus["CATEGORY"].str.contains("Radiology|ECG|Respiratory|Echo")==False]
# biased_notes_patients_corpus_filtered.to_csv("biased_notes_patients_corpus_filtered.csv")



   ROW_ID  SUBJECT_ID  HADM_ID   CHARTDATE           CHARTTIME  \
0  649967       86918   135138  2138-12-11 2138-12-11 17:37:00   
1  908036       19763     <NA>  2186-04-10 2186-04-10 00:00:00   
2  866000       21020     <NA>  2200-06-19 2200-06-19 00:00:00   
3  555551       49597   130164  2129-01-21 2129-01-21 17:19:00   
4  682095       62018   148182  2149-05-15 2149-05-15 13:52:00   

            STORETIME    CATEGORY DESCRIPTION   CGID  ISERROR  \
0 2138-12-11 18:02:24  Physician          CVI  16563     <NA>   
1                 NaT   Radiology         RVG   <NA>     <NA>   
2                 NaT   Radiology         RVG   <NA>     <NA>   
3 2129-01-21 17:19:13  Physician        CVICU  15626     <NA>   
4 2149-05-15 13:53:15  Physician        CVICU  15626     <NA>   

                                                TEXT  
0  TITLE:\n   CVICU\n   HPI:\n   66F s/p MVR(#31 ...  
1  RVG                                           ...  
2  RVG                                         

Explode by sentence, regex quote match

In [74]:
full_dataframe = biased_notes_patients_corpus_filtered
# Tokenize by sentence
nlp = English()  # just the language with no model

nlp.add_pipe('sentencizer')
full_dataframe["Sentence"] = full_dataframe["TEXT"].apply(lambda x: [sent.text for sent in nlp(x).sents])
full_dataframe = full_dataframe.explode("Sentence", ignore_index=True)
full_dataframe.rename(columns={"Unnamed: 0": "ROW_ID_new"}, inplace=True)
full_dataframe.index.name = "Sentence ID"

full_dataframe['Sentence'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)



# Regex matching all stemwords + any semantically similar word identified

In [82]:


doubt_word_list = pd.read_csv("doubt_word_list_round_2.csv")


# Create regex which is each stem word + all similar_words identified in the word embeddings step, which will be used to search the charts
def group_and_create_regex(dataframe):
    grouped_data = dataframe.groupby('stem_word')['similar_word'].apply(list).reset_index()
    grouped_data['regex'] = grouped_data.apply(lambda row: '|'.join([row['stem_word']] + row['similar_word']), axis=1)
    return grouped_data

grouped_data = group_and_create_regex(doubt_word_list)
print(grouped_data)

# Create function that takes regex from each row and searches the charts
def identify_matching_strings(full_dataframe, doubt_word_list):
    matching_dataframes = []

    for index, row in doubt_word_list.iterrows():
        regex = row['regex']
        stem_word = row['stem_word']
        matches = full_dataframe.loc[full_dataframe['Sentence'].str.contains(regex, flags=re.IGNORECASE)].copy()
        matches['regex'] = regex
        matches['stem_word'] = stem_word
        matching_dataframes.append(matches)

    if matching_dataframes:
        return pd.concat(matching_dataframes, axis=0)
    else:
        return pd.DataFrame()


result = identify_matching_strings(full_dataframe, grouped_data)
print(result)
result




,ROW_ID_x,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,...,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,Sentence,regex,stem_word,doubt_testimony,annotator_comments
Sentence ID,,,,,,,,,,,,,,,,,,,,,
31989,35891,99451,176459,2187-01-05,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,2120-06-15,2190-07-03,NaT,2190-07-03,1,They adamantly stated they wished to take him ...,adamant|adamantly|adamant/belligerant|addamant,adamant,,
51353,1515702,19213,122233,2204-07-27,2204-07-27 06:42:00,2204-07-27 06:52:00,Nursing/other,Report,15477,<NA>,...,2168-10-06,2206-06-20,NaT,2206-06-20,1,Surgery in to get her consent which she adaman...,adamant|adamantly|adamant/belligerant|addamant,adamant,,
63039,47892,20577,115141,2184-04-28,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,2125-05-16,2191-06-06,2191-06-06,NaT,1,Podiatry was consulted and recommended right ...,adamant|adamantly|adamant/belligerant|addamant,adamant,,
63089,47892,20577,115141,2184-04-28,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,2125-05-16,2191-06-06,2191-06-06,NaT,1,"However, the patient adamantly refused and di...",adamant|adamantly|adamant/belligerant|addamant,adamant,,
63443,1532531,20577,159034,2184-08-11,2184-08-11 17:14:00,2184-08-11 17:34:00,Nursing/other,Report,19613,<NA>,...,2125-05-16,2191-06-06,2191-06-06,NaT,1,Mult people spoke with pt today about need to...,adamant|adamantly|adamant/belligerant|addamant,adamant,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9101818,38819,88476,141309,2197-10-24,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,2135-05-02,NaT,NaT,NaT,0,He insists on taking warfarin (home dose 12.5m...,insists|insisting|reinsists|insist|insisted|.i...,insists,,
9104962,10039,83681,198636,2159-04-03,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,2095-05-07,NaT,NaT,NaT,0,She believes that this precipitated the suicid...,insists|insisting|reinsists|insist|insisted|.i...,insists,,
9107272,24357,70340,190099,2140-10-10,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,2089-12-25,NaT,NaT,NaT,0,He insisted upon leaving and began to walk off...,insists|insisting|reinsists|insist|insisted|.i...,insists,,


In [81]:

quoted_dataframe = result

quoted_dataframe["doubt_testimony"] = ""
quoted_dataframe["annotator_comments"] = ""
# 1459 matching sentences from total of 9,118,501 sentences from 366,002 charts
quoted_dataframe2 = quoted_dataframe.drop_duplicates(subset=['Sentence'])
quoted_dataframe2.to_csv("doubt_markers_dataframe_annotate.csv")



Explore the Data


In [ ]:
#from google.colab import runtime
#runtime.unassign()